In [1]:
# for loading the pre-trained models
from cryspnet.models import *
# for feature generation and plotting
from cryspnet.utils import *
# for accessing some pre-defined constant
from cryspnet.config import *

In [2]:
import pandas as pd
import numpy as np

# Load in the pre-trained models 

These three API helps you to retrieved them
```Python
    load_Bravais_models(n_ensembler, which)
    load_Lattice_models()
    load_SpaceGroup_models()
```

**Note**: If the file cannot be found error showed, make sure your follow the instruction to download and extract those models to the correct place.

In [3]:
BE = load_Bravais_models(n_ensembler=5, which="whole")

/home/aurora/anaconda3/envs/hallu/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning:

source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.

/home/aurora/anaconda3/envs/hallu/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning:

source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.

/home/aurora/anaconda3/envs/hallu/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning:

source code of class 'torch.nn.modules.batchnorm.BatchNorm1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set

In [4]:
LB = load_Lattice_models()

/home/aurora/anaconda3/envs/hallu/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning:

source code of class 'torch.nn.modules.loss.MSELoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.



In [5]:
SGB = load_SpaceGroup_models()

# Download the superhard dataset

Matminer provide an easy to access API for fast dataset retrieval. Check out [this page](https://hackingmaterials.lbl.gov/matminer/dataset_summary.html) for datasets that are all available

In [6]:
from matminer.datasets import load_dataset

In [7]:
shard = load_dataset("brgoch_superhard_training")

In [8]:
shard.head()

,formula,bulk_modulus,shear_modulus,composition,material_id,structure,brgoch_feats,suspect_value
0,AlPt3,225.230461,91.197748,"(Al, Pt)",mp-188,"[[0. 0. 0.] Al, [0. 1.961404 1.961404] P...","{'atomic_number_feat_1': 123.5, 'atomic_number...",False
1,Mn2Nb,232.696340,74.590157,"(Mn, Nb)",mp-12659,[[-2.237652e-08 1.429742e+00 5.926141e+00] M...,"{'atomic_number_feat_1': 45.5, 'atomic_number_...",False
2,HfO2,204.573433,98.564374,"(Hf, O)",mp-352,"[[2.244502 3.85793 4.833907] O, [2.778831 1.2...","{'atomic_number_feat_1': 44.0, 'atomic_number_...",False
3,Cu3Pt,159.312640,51.778816,"(Cu, Pt)",mp-12086,"[[0. 1.861442 1.861442] Cu, [1.861442 1....","{'atomic_number_feat_1': 82.5, 'atomic_number_...",False
4,Mg3Pt,69.637565,27.588765,"(Mg, Pt)",mp-18707,"[[0. 0. 2.736265] Mg, [0. 0....","{'atomic_number_feat_1': 57.0, 'atomic_number_...",False


# Generate Predictors

In [9]:
FG = FeatureGenerator()

In [10]:
ext_magpie = FG.generate(shard[['formula']])

# Prediction-High Level API

Utilize the library is simple, every model has a **.predicts()** method that take a dataframe with predictors as input and output Bravais Lattices, space group numbers or Lattice parameters. 

## Bravais Lattice

Prediction from Bravais Lattice Models has two parts in the output: First one is the top-n highest probability in the prediction. Second one is the classes that associated with those in the first one. The top-n is controlled by the parameter **topn_bravais**. The first column of 

In [11]:
probs, Bravais = BE.predicts(ext_magpie, topn_bravais=2)

The probability is order from left to right meaning the highest one are always on the left.

In [12]:
probs[:2]

array([[0.523457, 0.172805],
       [0.415681, 0.219132]], dtype=float32)

Here is the corresponding Bravais Lattice

In [13]:
Bravais[:2]

array([['tetragonal (P)', 'cubic (P)'],
       ['hexagonal (P)', 'cubic (F)']], dtype=object)

## Lattice Parameter and Space Group

To use those two model, a column with the name **Bravais** must exists in the input dataframe. This information could be obtained from previous predictions or experimental data.

In [14]:
lattices = []
spacegroups = []
for i in range(0, 2):
    ext_magpie['Bravais'] = Bravais[:, i]
    lattice = LB.predicts(ext_magpie)
    lattices.append(lattice)
    
    spacegroup = SGB.predicts(ext_magpie)
    spacegroups.append(spacegroup)

Lattice Parameter Model would output a dataframe contains a b c α, β,and γ

In [15]:
lattices[0].head()

,a,b,c,alpha,beta,gamma,v
0,4.912076,4.912076,8.154070,90.0,90.000000,90.0,196.745371
1,4.925382,4.925382,8.688627,90.0,90.000000,120.0,182.541526
2,4.756757,4.762517,5.409559,90.0,100.962353,90.0,120.312681
3,4.027474,4.027474,4.027474,90.0,90.000000,90.0,65.327840
4,7.697452,7.697452,9.074168,90.0,90.000000,120.0,465.619745


Space Group Model would output top-n probability and corresponding space group number in similar manner as Bravais Lattice Model

In [16]:
sg_probs, sg = spacegroups[0]

In [17]:
sg_probs[:5]

array([[9.349338e-01, 5.790174e-02, 3.068845e-03],
       [9.995158e-01, 1.539353e-04, 6.774997e-05],
       [9.930828e-01, 2.975656e-03, 2.040454e-03],
       [9.905792e-01, 6.351713e-03, 1.794986e-03],
       [7.484830e-01, 8.080608e-02, 3.511417e-02]])

In [18]:
sg[:5]

array([[127., 123.,  99.],
       [194., 158., 147.],
       [ 14.,  13.,   4.],
       [221., 213., 223.],
       [185., 194., 193.]])

## Low Level API

### Load dataset to the model

All models contain **.load()** method to attach the dataset to the model. The model then would perform pre-processing on the input dataset. 

In [19]:
BE.load(ext_magpie)
LB.load(ext_magpie) # note we have "Bravais" in this dataframe already
SGB.load(ext_magpie)

### Get the Raw Prediction
Sometimes, processing raw prediction is easy for further analysis. Make sure you load in dataset to the models before running lines in below.

The raw prediction is stored in the form of **torch tensor**. Prediction is stored in different format that is specified to each models. 

#### Bravais Lattice Model

For Bravais Lattice Model, raw prediction contains output from **N** (defined by user) models. i.e. the shape of the tensor is (N, dataset_size, 14)

In [20]:
preds = BE.get_preds()

In [21]:
preds.shape

torch.Size([5, 2574, 14])

The Bravais Lattice map to the predicted probability is stored in **.classes**

In [22]:
BE.classes

array(['cubic (F)', 'cubic (I)', 'cubic (P)', 'hexagonal (P)', 'monoclinic (C)', 'monoclinic (P)', 'orthorhombic (C)',
       'orthorhombic (F)', 'orthorhombic (I)', 'orthorhombic (P)', 'rhombohedral (P)', 'tetragonal (I)',
       'tetragonal (P)', 'triclinic (P)'], dtype=object)

#### Space Group Model and Lattice Parameter Model

For Space Group Model and Lattice Parameter Model, the raw prediction is a dictionary containing prediction from different sub-models. The key of the dictionary shows which sub-models the prediction comes from.

In [23]:
preds = SGB.get_preds()

In [24]:
preds.keys()

dict_keys(['cubic (F)', 'cubic (I)', 'cubic (P)', 'hexagonal (P)', 'monoclinic (C)', 'monoclinic (P)', 'orthorhombic (C)', 'orthorhombic (F)', 'orthorhombic (I)', 'orthorhombic (P)', 'rhombohedral (P)', 'tetragonal (I)', 'tetragonal (P)', 'triclinic (P)'])

In [25]:
preds['cubic (F)'][:3,]

tensor([[6.8972e-04, 4.9190e-04, 2.8339e-04, 5.6645e-06, 2.1340e-05, 6.4864e-03,
         8.1552e-05, 6.9725e-01, 1.3257e-04, 2.9449e-01, 6.7700e-05],
        [6.2911e-07, 9.3868e-06, 3.3187e-06, 2.5883e-07, 1.9014e-06, 1.5020e-03,
         4.4449e-07, 9.9832e-01, 6.2502e-06, 1.3960e-04, 1.2359e-05],
        [2.0791e-03, 3.0941e-02, 2.1696e-03, 2.0482e-04, 7.3345e-04, 8.6155e-01,
         5.3374e-04, 3.3889e-02, 8.3548e-04, 6.5142e-02, 1.9191e-03]])

The space group number for each sub model prediction is little more complicated. All submodels is stored in the **.Ps** field. For each sub-model access classes is similar to Bravais Lattice one. Here is an example of getting the classes for cubic (F) sub-model

In [26]:
SGB.Ps['cubic (F)'].classes

array([196, 202, 203, 209, 210, 216, 219, 225, 226, 227, 228])

In [27]:
preds = LB.get_preds()

In [28]:
preds.keys()

dict_keys(['cubic (F)', 'cubic (I)', 'cubic (P)', 'hexagonal (P)', 'monoclinic (C)', 'monoclinic (P)', 'orthorhombic (C)', 'orthorhombic (F)', 'orthorhombic (I)', 'orthorhombic (P)', 'rhombohedral (P)', 'tetragonal (I)', 'tetragonal (P)', 'triclinic (P)'])

In [29]:
preds['triclinic (P)'][:3]

tensor([[-1.1740, -2.0132, -2.3916,  0.1871,  0.5926, -0.4754],
        [-0.5988, -0.6773, -0.9498,  0.4634,  0.2637,  0.7202],
        [-0.5988, -0.6773, -0.9498,  0.4634,  0.2637,  0.7202]])

The **PRED_COLS** constant defined *config.py* tells you which lattice parameters the sub model is predicting.  

In [30]:
PRED_COLS

{'cubic (P)': ['a'],
 'cubic (F)': ['a'],
 'cubic (I)': ['a'],
 'monoclinic (P)': ['a', 'b', 'c', 'beta'],
 'monoclinic (C)': ['a', 'b', 'c', 'beta'],
 'hexagonal (P)': ['a', 'c'],
 'rhombohedral (P)': ['a', 'alpha'],
 'tetragonal (P)': ['a', 'c'],
 'tetragonal (I)': ['a', 'c'],
 'orthorhombic (P)': ['a', 'b', 'c'],
 'orthorhombic (C)': ['a', 'b', 'c'],
 'orthorhombic (F)': ['a', 'b', 'c'],
 'orthorhombic (I)': ['a', 'b', 'c'],
 'triclinic (P)': ['a', 'b', 'c', 'alpha', 'beta', 'gamma']}